In [11]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, BaseMessage
from typing import TypedDict, Annotated
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
load_dotenv()
from langgraph.checkpoint.memory import MemorySaver

class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",  # or another from above
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

def chat_node(state: State):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}  # wrapped in list to append properly

graph = StateGraph(State)
graph.add_node("chat_node", chat_node)

graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)
check_point=MemorySaver()
app = graph.compile(checkpointer=check_point)

In [12]:
thread_id = '1'

while True:
    user = input("Type here: ")
    print("user",user)
    if user.strip().lower() in ['end', 'stop']:
        break

    # Thread-based config for LangGraph
    conf = {'configurable': {'thread_id': thread_id}}

    # Send user message to chatbot
    response = app.invoke({'messages': [HumanMessage(content=user)]}, config=conf)

    # Print last assistant reply
    print("AI:", response['messages'][-1].content)

user where delhi stitutaed
AI:  Delhi, the capital city of India, is home to many prestigious educational institutions. Here are some of the well-known universities and colleges in Delhi:

1. University of Delhi: It is a central university located in New Delhi and is one of the largest universities in India with 86 departments, 77 colleges, and 5 other recognized institutions spread across the city.
2. Jawaharlal Nehru University (JNU): It is a public central university located in New Delhi and is known for its high-quality research and teaching in various fields.
3. Indian Institute of Technology (IIT) Delhi: It is one of the premier engineering institutions in India and is located in Hauz Khas, South Delhi.
4. All India Institute of Medical Sciences (AIIMS): It is a public medical college and medical research public university based in New Delhi.
5. Delhi Technological University (DTU): It is a public engineering university located in Delhi and is known for its undergraduate and post